$\Large\textbf{Lab 4.}$ $\large\textbf{Exercise 1.}$



In [1]:
import numpy as np 
from numpy.linalg import cond
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt

In [2]:
#Now we will define a Python function which will compute and return the objective function value 
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  return 400*x[0]**2 + 0.004*x[1]**4

In [3]:
#Now we will define a Python function which will compute and return the gradient value as a numpy array 
def evalg(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the gradient value
  #compute the gradient value and return it 
  return np.array([800*x[0],0.016*x[1]**3])

In [4]:
#method to find Hessian matrix: Complete the code
def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  return np.array([[800,0],[0,0.048*x[1]**2]])

In [5]:
#The method defines a way to construct D_k matrix used in scaling the gradient in the modified gradient descent scheme
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k
  a=1/(evalh(x)[0][0])
  b=1/(evalh(x)[1][1])
  return np.array([[a, 0], [0, b]])


In [6]:
def compute_D_k_newton(x):
    assert type(x) is np.ndarray
    assert len(x) == 2
    
    return np.linalg.inv(evalh(x))

In [7]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma, *args): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  #Complete the code 
  alpha = alpha_start
  pk = -gradf
  while evalf(np.add(x, alpha*pk)) > np.add(evalf(x), gamma*alpha*np.dot(gradf.T,pk)):
    alpha = rho*alpha

  return alpha
  

In [8]:
def compute_steplength_backtracking_scaled_direction(x, gradf, direction, alpha_start, rho, gamma, *args): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(direction) is np.ndarray and len(direction) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  
  
  #Complete the code 
  alpha = alpha_start
  pk = -gradf
  while evalf(np.add(x, alpha*np.dot(direction,pk))) > np.subtract(evalf(x), gamma*alpha*np.dot(np.dot(direction,gradf), gradf)):
    alpha = rho*alpha
  return alpha

In [9]:
#line search type 
CONSTANT_STEP_LENGTH = 1
BACKTRACKING_LINE_SEARCH = 2

# D_k choice type
newton = 3
diagonal_approx = 4

In [10]:
# Code for gradient descent with scaling to find the minimizer without scaling

def find_minimizer_gd(start_x, tol, line_search_type, *args):
    #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
    assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg(x)

    #initialization for backtracking line search
    if(line_search_type == BACKTRACKING_LINE_SEARCH):
        alpha_start = args[0]
        rho = args[1]
        gamma = args[2]

    k = 0

    while (np.linalg.norm(g_x) > tol): 
    
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps   
        x = np.subtract(x, np.multiply(step_length,g_x))
        k += 1 
        g_x = evalg(x) 

       # print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
    #print(k)
    return x, evalf(x), k
 

In [11]:
# Code for gradient descent with scaling to find the minimizer with scaling

def find_minimizer_gdscaling(start_x, tol, line_search_type, D_k_type, *args):
    #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
    assert type(start_x) is np.ndarray and len(start_x) == 2 
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg(x)
    
    #initialization for backtracking line search
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]

    k = 0

    while (np.linalg.norm(g_x) > tol): 
        if D_k_type == newton:
            d_k = compute_D_k_newton(x)
        elif D_k_type == diagonal_approx:
            d_k = compute_D_k(x)
        else:
            raise ValueError("Invalid argument.")


        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking_scaled_direction(x, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps
        x = np.subtract(x, np.multiply(step_length,np.dot(d_k, g_x))) 
        k += 1 
        g_x = evalg(x) 
        #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))


    return x, evalf(x), k
 

Code for answer 3

In [12]:
my_start_x = np.array([2.,2.])
my_tol= 1e-9
alpha_start = 1.
rho = 0.5
gamma = 0.5

In [13]:
#check gradient descent with backtracking line search 
x_opt_bls, f_bls, iter_bls = find_minimizer_gdscaling(my_start_x, my_tol, CONSTANT_STEP_LENGTH, newton, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls)
print("Minimum function value =", f_bls)
print("No of iterations =", iter_bls)

Minimizer = [0.         0.00304488]
Minimum function value = 3.4382653805813626e-13
No of iterations = 16


In [14]:
#check gradient descent with scaling and backtracking line search 
x_opt_bls_scaling, f_bls_scaling, iter_bls_scaling = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, newton, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls_scaling)
print("Minimum function value =", f_bls_scaling)
print("No of iterations =", iter_bls_scaling)

Minimizer = [0.         0.00304488]
Minimum function value = 3.4382653805813626e-13
No of iterations = 16


Code for answer 4

In [15]:
#check gradient descent with scaling and backtracking line search 
x_opt_bls_scaling, f_bls_scaling, iter_bls_scaling = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, diagonal_approx, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls_scaling)
print("Minimum function value =", f_bls_scaling)
print("No of iterations =", iter_bls_scaling)

Minimizer = [0.         0.00304488]
Minimum function value = 3.4382653805813626e-13
No of iterations = 16


In [18]:
my_start_x = np.array([2.,2.])
my_tol= 1e-6
alpha_start = 1.
rho = 0.5
gamma = 0.5

In [19]:
#check gradient descent with backtracking line search 
x_opt_bls, f_bls, iter_bls = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print("Minimizer =", x_opt_bls)
print("Minimum function value =", f_bls)
print("No of iterations =", iter_bls)

Minimizer = [3.53148711e-10 3.91377499e-02]
Minimum function value = 9.385197325158433e-09
No of iterations = 5707240


Answer 3) 

**Newton's method (with constant step length of 1)**

Minimizer = [0.0 , 0.00304488]

Minimum function value = 3.4382653805813626e-13

No of iterations = 16

**Newton's method (with Backtracking line search)**

Minimizer = [0.0 , 0.00304488]

Minimum function value = 3.4382653805813626e-13

No of iterations = 16

**Observations:** 

In both the cases, the minimizer is [0.0 , 0.00304488], the minimum function value is close to 0 (3.4382653805813626e-13) and the number of iterations is 16. We may say that the choice of the step-length did not affect the rate of convergence of the algorithm, rather the consideration of the scaling algorithm (with the scaling matrix taken as ($\nabla^2 f(\mathbf{x}))^{-1}$ in Newton's method) was more crucial.

Answer 4) 

**Gradient Descent (with scaling using diagonal matrix)**

Minimizer = [0.0 , 0.00304488]

Minimum function value = 3.4382653805813626e-13

No of iterations = 16

**Gradient Descent (without scaling)** 

(Here, tolerance was taken to be $10^{-6}$ because it was taking a long time to converge)

Minimizer = [3.53148711e-10 3.91377499e-02]

Minimum function value = 9.385197325158433e-09

No of iterations = 5707240

**Observations:** 

Graident Descent with scaling using diagonal matrix uses very less iterations (16) when compared to gradient descent without scaling (5707240 iterations) even though the tolerance levels in both are $10^{-9}$ and $10^{-6}$ respectively. Gradient Descent with scaling using diagonal matrix gave a better result (Minimizer close to (0,0) and minimum function value close to 0) than gradient descent without scaling.
